In [1]:
import pandas as pd
import numpy as np
dfs = pd.read_csv("full_df.csv")
#去掉年龄太大或者太小的
dfs = dfs[dfs['Patient Age'].map(lambda x:10<=x<=80)]

In [2]:
def has_glaucoma(text):
    if "glaucoma" in text:
        return 1
    else:
        return 0
dfs["left_glaucoma"] = dfs["Left-Diagnostic Keywords"].apply(lambda x: has_glaucoma(x))
dfs["right_glaucoma"] = dfs["Right-Diagnostic Keywords"].apply(lambda x: has_glaucoma(x))
left_glaucoma = dfs.loc[(dfs.G ==1) & (dfs.left_glaucoma == 1)]["Left-Fundus"].values
right_glaucoma = dfs.loc[(dfs.G ==1) & (dfs.right_glaucoma == 1)]["Right-Fundus"].values
def has_retinopathy(text):
    if "retinopathy" in text:
        return 1
    else:
        return 0
dfs["left_retinopathy"] = dfs["Left-Diagnostic Keywords"].apply(lambda x: has_retinopathy(x))
dfs["right_retinopathy"] = dfs["Right-Diagnostic Keywords"].apply(lambda x: has_retinopathy(x))
left_retinopathy = dfs.loc[(dfs.D ==1) & (dfs.left_retinopathy == 1)]["Left-Fundus"].sample(300,random_state=42).values
right_retinopathy = dfs.loc[(dfs.D ==1) & (dfs.right_retinopathy == 1)]["Right-Fundus"].sample(300,random_state=42).values
def has_cataract(text):
    if "cataract" in text:
        return 1
    else:
        return 0
dfs["left_cataract"] = dfs["Left-Diagnostic Keywords"].apply(lambda x: has_cataract(x))
dfs["right_cataract"] = dfs["Right-Diagnostic Keywords"].apply(lambda x: has_cataract(x))
left_cataract = dfs.loc[(dfs.C ==1) & (dfs.left_cataract == 1)]["Left-Fundus"].values
right_cataract = dfs.loc[(dfs.C ==1) & (dfs.right_cataract == 1)]["Right-Fundus"].values   

glaucoma = np.concatenate((left_glaucoma,right_glaucoma),axis=-1)
retinopathy = np.concatenate((left_retinopathy,right_retinopathy),axis=-1)
cataract = np.concatenate((left_cataract,right_cataract),axis=-1)

In [3]:
# 创建标签数组
labels = np.concatenate((np.zeros(len(glaucoma)), np.ones(len(retinopathy)), np.full(len(cataract), 2)))
labels = labels.astype(int)
# 创建数据数组
data = np.concatenate((glaucoma, retinopathy, cataract))

# 创建DataFrame
df = pd.DataFrame({'Data': data, 'Label': labels})
df.tail(5)

,Data,Label
1708,2243_right.jpg,2
1709,2246_right.jpg,2
1710,2247_right.jpg,2
1711,2248_right.jpg,2
1712,2282_right.jpg,2


In [4]:
res = df.sample(n = len(df))
res

,Data,Label
652,4312_left.jpg,1
44,1264_left.jpg,0
275,1440_left.jpg,0
753,26_left.jpg,1
368,1266_right.jpg,0
...,...,...
1219,1164_left.jpg,2
450,1963_right.jpg,0
692,667_left.jpg,1
730,4613_left.jpg,1


In [5]:
train = res.iloc[:int(len(res)*0.7)]
train

,Data,Label
652,4312_left.jpg,1
44,1264_left.jpg,0
275,1440_left.jpg,0
753,26_left.jpg,1
368,1266_right.jpg,0
...,...,...
546,1392_right.jpg,0
1600,726_right.jpg,2
253,1396_left.jpg,0
866,763_left.jpg,1


In [6]:
val = res.iloc[int(len(res)*0.7):int(len(res)*0.9)]
val

,Data,Label
833,4504_left.jpg,1
1698,2228_right.jpg,2
725,688_left.jpg,1
632,1718_left.jpg,1
24,1226_left.jpg,0
...,...,...
490,1239_right.jpg,0
110,1433_left.jpg,0
88,1386_left.jpg,0
261,1414_left.jpg,0


In [7]:
test = res.iloc[int(len(res)*0.9):]
test

,Data,Label
281,1458_left.jpg,0
1338,611_left.jpg,2
1609,1167_right.jpg,2
1119,4203_right.jpg,1
401,1345_right.jpg,0
...,...,...
1219,1164_left.jpg,2
450,1963_right.jpg,0
692,667_left.jpg,1
730,4613_left.jpg,1


In [8]:
dfh = train[train['Label'] == 0]
dfh

,Data,Label
44,1264_left.jpg,0
275,1440_left.jpg,0
368,1266_right.jpg,0
266,1424_left.jpg,0
23,1225_left.jpg,0
...,...,...
85,1374_left.jpg,0
214,1277_left.jpg,0
524,1306_right.jpg,0
546,1392_right.jpg,0


In [9]:
target = '1475_left'
is_present = dfh['Data'].apply(lambda x: target in x).any()
is_present

False

In [10]:
import shutil
for i in range(len(dfh)):
    file = dfh['Data'].iloc[i]
    #路径1
    file1 = 'preprocessed_images/'+file
    #路径2
    file2 = 'datas2/train/Glaucoma/'+'Glaucoma_'+str(i)+'.jpg'
    try:
        # 进行copy
        shutil.copy(file1, file2)
    except FileNotFoundError:
        # 处理文件不存在的异常（跳过错误）
        continue

In [11]:
dfh = train[train['Label'] == 1]
print(dfh)
import shutil
for i in range(len(dfh)):
    file = dfh['Data'].iloc[i]
    #路径1
    file1 = 'preprocessed_images/'+file
    #路径2
    file2 = 'datas2/train/Retinopathy/'+'Retinopathy_'+str(i)+'.jpg'
    try:
        # 进行copy
        shutil.copy(file1, file2)
    except FileNotFoundError:
        # 处理文件不存在的异常（跳过错误）
        continue

                Data  Label
652    4312_left.jpg      1
753      26_left.jpg      1
1105  3985_right.jpg      1
638     263_left.jpg      1
621     981_left.jpg      1
...              ...    ...
724    4179_left.jpg      1
1116  1364_right.jpg      1
618     993_left.jpg      1
1021  4193_right.jpg      1
866     763_left.jpg      1

[427 rows x 2 columns]


In [12]:
dfh = train[train['Label'] == 2]
print(dfh)
import shutil
for i in range(len(dfh)):
    file = dfh['Data'].iloc[i]
    #路径1
    file1 = 'preprocessed_images/'+file
    #路径2
    file2 = 'datas2/train/Cataract/'+'Cataract_'+str(i)+'.jpg'
    try:
        # 进行copy
        shutil.copy(file1, file2)
    except FileNotFoundError:
        # 处理文件不存在的异常（跳过错误）
        continue

                Data  Label
1475   970_right.jpg      2
1393   2140_left.jpg      2
1520  2142_right.jpg      2
1289   2195_left.jpg      2
1471   726_right.jpg      2
...              ...    ...
1423   2192_left.jpg      2
1691  2219_right.jpg      2
1437   2216_left.jpg      2
1600   726_right.jpg      2
1634  2126_right.jpg      2

[373 rows x 2 columns]


In [13]:
dfh = val[val['Label'] == 0]
print(dfh)
import shutil
for i in range(len(dfh)):
    file = dfh['Data'].iloc[i]
    #路径1
    file1 = 'preprocessed_images/'+file
    #路径2
    file2 = 'datas2/val/Glaucoma/'+'Glaucoma_'+str(i)+'.jpg'
    try:
        # 进行copy
        shutil.copy(file1, file2)
    except FileNotFoundError:
        # 处理文件不存在的异常（跳过错误）
        continue

               Data  Label
24    1226_left.jpg      0
66    1301_left.jpg      0
523  1302_right.jpg      0
352  1238_right.jpg      0
40    1258_left.jpg      0
..              ...    ...
465   746_right.jpg      0
490  1239_right.jpg      0
110   1433_left.jpg      0
88    1386_left.jpg      0
261   1414_left.jpg      0

[134 rows x 2 columns]


In [14]:
dfh = val[val['Label'] == 1]
print(dfh)
import shutil
for i in range(len(dfh)):
    file = dfh['Data'].iloc[i]
    #路径1
    file1 = 'preprocessed_images/'+file
    #路径2
    file2 = 'datas2/val/Retinopathy/'+'Rentinopathy_'+str(i)+'.jpg'
    try:
        # 进行copy
        shutil.copy(file1, file2)
    except FileNotFoundError:
        # 处理文件不存在的异常（跳过错误）
        continue

               Data  Label
833   4504_left.jpg      1
725    688_left.jpg      1
632   1718_left.jpg      1
972  4570_right.jpg      1
665   4084_left.jpg      1
..              ...    ...
643    498_left.jpg      1
768   4181_left.jpg      1
980  4105_right.jpg      1
756   1106_left.jpg      1
868   4455_left.jpg      1

[112 rows x 2 columns]


In [15]:
dfh = val[val['Label'] == 2]
print(dfh)
import shutil
for i in range(len(dfh)):
    file = dfh['Data'].iloc[i]
    #路径1
    file1 = 'preprocessed_images/'+file
    #路径2
    file2 = 'datas2/val/Cataract/'+'Cataract_'+str(i)+'.jpg'
    try:
        # 进行copy
        shutil.copy(file1, file2)
    except FileNotFoundError:
        # 处理文件不存在的异常（跳过错误）
        continue

                Data  Label
1698  2228_right.jpg      2
1420   2183_left.jpg      2
1335    553_left.jpg      2
1287   2192_left.jpg      2
1576  2233_right.jpg      2
...              ...    ...
1406   2159_left.jpg      2
1246   2124_left.jpg      2
1222   1444_left.jpg      2
1667  2181_right.jpg      2
1431   2206_left.jpg      2

[96 rows x 2 columns]


In [17]:
dfh = test[test['Label'] == 0]
print(dfh)
print(dfh.shape)
import shutil
for i in range(len(dfh)):
    file = dfh['Data'].iloc[i]
    #路径1
    file1 = 'preprocessed_images/'+file
    #路径2
    file2 = 'datas2/test/Glaucoma/'+'Glaucoma_'+str(i)+'.jpg'
    try:
        # 进行copy
        shutil.copy(file1, file2)
    except FileNotFoundError:
        # 处理文件不存在的异常（跳过错误）
        continue

               Data  Label
281   1458_left.jpg      0
401  1345_right.jpg      0
472  1214_right.jpg      0
463   365_right.jpg      0
243   1353_left.jpg      0
400  1344_right.jpg      0
108   1429_left.jpg      0
143   1515_left.jpg      0
379  1284_right.jpg      0
67    1302_left.jpg      0
129   1479_left.jpg      0
362  1259_right.jpg      0
252   1392_left.jpg      0
448  1881_right.jpg      0
161    167_left.jpg      0
82    1353_left.jpg      0
199   1251_left.jpg      0
211   1271_left.jpg      0
236   1327_left.jpg      0
184   1226_left.jpg      0
242   1352_left.jpg      0
274   1439_left.jpg      0
363  1260_right.jpg      0
425  1433_right.jpg      0
386  1294_right.jpg      0
77    1334_left.jpg      0
426  1434_right.jpg      0
99    1412_left.jpg      0
558  1434_right.jpg      0
578  1507_right.jpg      0
34    1246_left.jpg      0
32    1239_left.jpg      0
342  1222_right.jpg      0
563  1458_right.jpg      0
535  1351_right.jpg      0
343  1225_right.jpg      0
2

In [18]:
dfh = test[test['Label'] == 1]
print(dfh)
print(dfh.shape)
import shutil
for i in range(len(dfh)):
    file = dfh['Data'].iloc[i]
    #路径1
    file1 = 'preprocessed_images/'+file
    #路径2
    file2 = 'datas2/test/Retinopathy/'+'Retinopathy_'+str(i)+'.jpg'
    try:
        # 进行copy
        shutil.copy(file1, file2)
    except FileNotFoundError:
        # 处理文件不存在的异常（跳过错误）
        continue

                Data  Label
1119  4203_right.jpg      1
1029  4342_right.jpg      1
808     713_left.jpg      1
1176  4495_right.jpg      1
648    4081_left.jpg      1
...              ...    ...
770    4074_left.jpg      1
667    3943_left.jpg      1
692     667_left.jpg      1
730    4613_left.jpg      1
691    4190_left.jpg      1

[61 rows x 2 columns]
(61, 2)


In [19]:
dfh = test[test['Label'] == 2]
print(dfh)
print(dfh.shape)
import shutil
for i in range(len(dfh)):
    file = dfh['Data'].iloc[i]
    #路径1
    file1 = 'preprocessed_images/'+file
    #路径2
    file2 = 'datas2/test/Cataract/'+'Cataract_'+str(i)+'.jpg'
    try:
        # 进行copy
        shutil.copy(file1, file2)
    except FileNotFoundError:
        # 处理文件不存在的异常（跳过错误）
        continue

                Data  Label
1338    611_left.jpg      2
1609  1167_right.jpg      2
1601   769_right.jpg      2
1202    560_left.jpg      2
1367   2101_left.jpg      2
1490  2095_right.jpg      2
1577  2234_right.jpg      2
1307   2226_left.jpg      2
1408   2164_left.jpg      2
1459   2262_left.jpg      2
1391   2137_left.jpg      2
1553  2197_right.jpg      2
1472   769_right.jpg      2
1659  2165_right.jpg      2
1701  2233_right.jpg      2
1311   2233_left.jpg      2
1678  2197_right.jpg      2
1524  2151_right.jpg      2
1425   2195_left.jpg      2
1257   2138_left.jpg      2
1547  2191_right.jpg      2
1399   2151_left.jpg      2
1542  2181_right.jpg      2
1597   553_right.jpg      2
1448   2234_left.jpg      2
1564  2216_right.jpg      2
1549  2193_right.jpg      2
1661  2168_right.jpg      2
1630  2114_right.jpg      2
1272   2164_left.jpg      2
1230   2098_left.jpg      2
1573  2228_right.jpg      2
1210    809_left.jpg      2
1689  2216_right.jpg      2
1401   2153_left.jpg